In [400]:
import pandas as pd
import ast
import mysql.connector

# Database connection settings
db_config = {
    "host": "localhost",
    "user": "root",
    "password": "root",
    "database": "tjopdb",
}

conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

## Colors Table

1. Retrieve and sanitize colors

In [401]:
csv_file_path = "data/The Joy Of Painiting - Colors Used"
df = pd.read_csv(csv_file_path)

# Create an empty dictionary to store the color-hex mappings
color_dict = {}

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    colors_list = ast.literal_eval(row['colors'])
    hex_list = ast.literal_eval(row['color_hex'])

    cleaned_colors_list = [color.replace('\r', '').replace('\n', '').lower() for color in colors_list]

    # Add each color-hex pair to the dictionary
    for color, hex_code in zip(cleaned_colors_list, hex_list):
        color_dict[color] = hex_code

# Print the dictionary
# print(color_dict)


2. Connect to database and store colors in colors table

In [402]:
for key, val in color_dict.items():
    insert_query = f"""
        INSERT IGNORE INTO Colors (name, hex)
        VALUES ('{key.lower()}', '{val}')
    """
    cursor.execute(insert_query)

conn.commit()


print("CSV data has been transferred to the MySQL database.")


CSV data has been transferred to the MySQL database.


## Episodes Table

1. Retrieve and sanitize episode information

*Note that the following code uses the unsafe assumption that the files are in order and that the rows in each file correspond to each other.  The "safe" code is also provided, which outputs any mismatching rows and leaves blanks in the date field where rows are mismatched*

In [403]:
from datetime import datetime

file_path = "data/The Joy Of Painting - Episode Dates"

# date_dict = {} # SAFE
date_list = [] # UNSAFE
with open(file_path, "r") as file:
    line = file.readline()
    while line:
        # Do stuff with the line
        start_quote = line.find('"') + 1
        end_quote = line.find('"', start_quote)

        # title = line[start_quote:end_quote].lower()
        title = line[start_quote:end_quote]

        start_date = line.find('(', end_quote) + 1
        end_date = line.find(')', start_date)

        date = line[start_date:end_date]

        date_object = datetime.strptime(date, "%B %d, %Y")
        formatted_date = date_object.strftime("%Y-%m-%d")
        # SAFE: Use this to be more
        # date_dict[title] = formatted_date
        # UNSAFE: Used because we know the files are in order and correspond to
        # each other.
        date_list.append(formatted_date)

        line = file.readline()

# FOR TESTING
# print("Date Dict:")
# for key, val in date_dict.items():
#     print(f'{key}: {val}')
# print("\n\n")

# Convert the painting titles in the DataFrame to lowercase and remove leading/trailing spaces
# df['painting_title'] = df['painting_title'].str.strip().str.lower()
df['painting_title'] = df['painting_title'].str.strip()

mismatch = {}

# Populate the air_date column using the cleaned painting titles
for index, row in df.iterrows():
    painting_title = row['painting_title']
    # SAFE
    # if painting_title in date_dict:
        # df.at[index, 'air_date'] = date_dict[painting_title]
    # else:
    #     print(f"Missing date for painting: {painting_title}")
    df.at[index, 'air_date'] = date_list[index] # UNSAFE

# Remove unwanted columns from df
desired_columns = ['painting_title', 'season', 'episode', 'youtube_src', 'air_date', 'img_src']
episodes_df = df.loc[:, desired_columns].copy()

# FOR USE LATER
desired_columns = ['painting_title', 'colors']
pc_df = df.loc[:, desired_columns].copy()


# Print the updated DataFrame
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
df_head = episodes_df.head(5)
df_head

,painting_title,season,episode,youtube_src,air_date,img_src
0,A Walk in the Woods,1,1,https://www.youtube.com/embed/oh5p5f5_-7A,1983-01-11,https://www.twoinchbrush.com/images/painting28...
1,Mt. McKinley,1,2,https://www.youtube.com/embed/RInDWhYceLU,1983-01-11,https://www.twoinchbrush.com/images/painting28...
2,Ebony Sunset,1,3,https://www.youtube.com/embed/UOziR7PoVco,1983-01-18,https://www.twoinchbrush.com/images/painting28...
3,Winter Mist,1,4,https://www.youtube.com/embed/0pwoixRikn4,1983-01-25,https://www.twoinchbrush.com/images/painting28...
4,Quiet Stream,1,5,https://www.youtube.com/embed/DFSIQNjKRfk,1983-02-01,https://www.twoinchbrush.com/images/painting28...


2. Connect to database and store episodes in episodes table.

In [404]:
insert_query = f"""
    INSERT INTO Episodes (title, season, episode, youtube_src, air_date, img_src)
    VALUES (%s, %s, %s, %s, %s, %s)
"""

for row in episodes_df.itertuples(index=False):
    cursor.execute(insert_query, row)
    episode_id = cursor.lastrowid

conn.commit()

print("CSV data has been transferred to the MySQL database.")

CSV data has been transferred to the MySQL database.


## Subjects Table

1. Retrieve and sanitize Subject information

In [405]:
csv_file_path = "data/The Joy Of Painiting - Subject Matter"
subjects_df = pd.read_csv(csv_file_path)

subjects_df.columns = subjects_df.columns.str.strip()

id_vars = ['EPISODE', 'TITLE']
value_vars = subjects_df.columns[2:]

subjects_df = pd.melt(subjects_df, id_vars=id_vars, value_vars=value_vars, var_name='SUBJECT', value_name='VALUE')
subjects_df = subjects_df[subjects_df['VALUE'] == 1]
subjects_df = subjects_df.drop(columns=['VALUE', 'EPISODE'])
subjects_df['TITLE'] = subjects_df['TITLE'].str.strip('"')
subjects_df = subjects_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


episodes_df['painting_title'] = episodes_df['painting_title'].str.lower()
episodes_df.head()

subjects_df = pd.merge(episodes_df, subjects_df, left_on='painting_title', right_on='TITLE', how='inner')
subjects_df = subjects_df[['painting_title', 'SUBJECT']]

# NEW
current_title = ''
title_counter = 0

# Iterate over the rows of the DataFrame
for index, row in subjects_df.iterrows():
    if row['painting_title'] != current_title:
        title_counter += 1
        current_title = row['painting_title']
    subjects_df.at[index, 'title_number'] = title_counter

# Remove the original 'painting_title' column if needed
subjects_df = subjects_df.drop(columns='painting_title')
# Convert the 'title_number' column to integer data type
subjects_df['title_number'] = subjects_df['title_number'].astype(int)

# Rename columns
subjects_df = subjects_df.rename(columns={'SUBJECT': 'subject_name', 'title_number': 'episode_id'})
subjects_df = subjects_df[['episode_id', 'subject_name']]
# END OF NEW

subjects_df.head()



,episode_id,subject_name
0,1,bushes
1,1,deciduous
2,1,grass
3,1,river
4,1,tree


2. Add data to Subjects table in database

In [406]:
# Remove duplicate entries from the dataframe
subjects_df = subjects_df.drop_duplicates(subset=['episode_id', 'subject_name'])

# Prepare the SQL statement for inserting data into the Subjects table
sql = "INSERT INTO Subjects (episode_id, subject_name) VALUES (%s, %s)"

# Convert the DataFrame to a list of tuples
values = subjects_df.to_records(index=False).tolist()

# Execute the SQL statement to insert the data into the Subjects table
cursor.executemany(sql, values)

# Commit the changes to the database
conn.commit()


## PaintingColors Table

In [407]:
from ast import literal_eval

# Assuming you have the 'pc_df' DataFrame with the 'colors' column containing list-like strings

# Convert string representations of lists to actual lists
pc_df['colors'] = pc_df['colors'].apply(lambda x: literal_eval(x) if pd.notna(x) else x)

# Remove \n and \r from the 'colors' column (if any)
pc_df['colors'] = pc_df['colors'].apply(lambda x: [color.replace('\n', '').replace('\r', '') for color in x] if isinstance(x, list) else x)

# Explode the DataFrame and overwrite the original
pc_df = pc_df.explode('colors')


# replace names with ids
# Create a mapping of unique values in 'painting_title' to integers
mapping = {title: i + 1 for i, title in enumerate(pc_df['painting_title'].unique())}

# Replace values in 'painting_title' with their corresponding integers
pc_df['painting_title'] = pc_df['painting_title'].replace(mapping)

# Reorder columns if needed (optional step)
pc_df = pc_df[['painting_title', 'colors']]
pc_df.rename(columns={'painting_title': 'episode_id', 'colors': 'color_name'}, inplace=True)

# Remove duplicate entries from the dataframe
pc_df = pc_df.drop_duplicates(subset=['episode_id', 'color_name'])
pc_df['color_name'] = pc_df['color_name'].str.lower()

# Print the DataFrame
print(pc_df)

     episode_id        color_name
0             1  alizarin crimson
0             1        bright red
0             1    cadmium yellow
0             1     phthalo green
0             1     prussian blue
0             1         sap green
0             1    titanium white
0             1    van dyke brown
1             2  alizarin crimson
1             2        bright red
1             2    cadmium yellow
1             2     phthalo green
1             2     prussian blue
1             2         sap green
1             2    titanium white
1             2    van dyke brown
2             3  alizarin crimson
2             3       black gesso
2             3        bright red
2             3    cadmium yellow
2             3     phthalo green
2             3     prussian blue
2             3         sap green
2             3    titanium white
2             3    van dyke brown
3             4     prussian blue
3             4    titanium white
3             4    van dyke brown
4             

In [413]:
cleaned_colors_list

['alizarin crimson',
 'black gesso',
 'bright red',
 'cadmium yellow',
 'dark sienna',
 'indian yellow',
 'midnight black',
 'phthalo blue',
 'sap green',
 'titanium white',
 'van dyke brown',
 'yellow ochre']

In [409]:
# # Prepare the SQL statement for inserting data into the Subjects table
# sql = "INSERT INTO Subjects (episode_id, color_id) VALUES (%s, %s)"

# # Convert the DataFrame to a list of tuples
# values = pc_df.to_records(index=False).tolist()

# # Execute the SQL statement to insert the data into the Subjects table
# cursor.executemany(sql, values)

# # Commit the changes to the database
# conn.commit()

In [410]:
cursor.close()
conn.close()